<a href="https://colab.research.google.com/github/kkkonts/coursera-matstat-AB/blob/main/HW07_estimates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#  НесмУщённый и состоятельный записыватель курсов

__Это задание является дополнительным и необязательным для решения.__

Филипп записывает онлайн-курс по статистике. Он ориентируется на то, что каждая неделя курса должна идти около $100$ минут. В голове у Филиппа есть оцениватель времени. Он оценивает по длительности каждое видео и записыват свои прикидки на бумагу. Когда видео снято, Филипп смотрит как долго оно длилось и записывает итоговый результат на бумагу. В таблице `filipp_course.tsv` представлены записи Филиппа за $11$ отснятых недель.

__Описание колонок:__

- `real_week` - хронологический номер недели
- `week` - какой по порядку записывалась неделя
- `type` - тип видео (доска, презентация, код)
- `estimate_time` - во сколько минут Филипп оценил время видео при разработке материала
- `real_time` - сколько реально длится это видео


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("filipp_course.tsv", sep='\t')
print(df.shape)
df.head()

(129, 5)


,real_week,week,type,estimate_time,real_time
0,6,1,blackboard,6,11.0
1,6,1,blackboard,10,7.0
2,6,1,blackboard,10,11.5
3,6,1,blackboard,5,6.0
4,6,1,presentation,5,3.0


Будем считать, что на протяжении всей записи у Филиппа в голове был один и тот же оцениватель (estimator). По этому оценивателю Филипп делал точечные оценки для каждого видео независимо друг от друга. Давайте поисследуем оцениватель Филиппа на различные статистические свойства.

__а)__ Посчитайте разницу между `estimate_time` и `real_time`. Запишите её в новую колонку `bias`. Это то, насколько каждая оценка Филиппа оказалась смещена. Посчитайте среднее смещение, запишите его в переменную `bias_mean`. Проинтерпретируйте получившееся число. Правда ли, что Филипп делает несмещённые оценки?

In [3]:

df['bias'] = df['estimate_time'] - df['real_time']
bias_mean = df['bias'].mean()
bias_mean

-2.1790697674418604

In [4]:
# проверка того, что задание решено верно (задание необязательно - все тесты открыты)
assert np.abs(bias_mean  + 2.179) < 1e-2


__б)__ Посчитайте среднее смещение, которое делает Филипп для каждого типа контента. Для какого типа смещение оказывается самым низким? Запишите его в переменную `bias_min`.

In [10]:
df_gr = df.groupby(['type'])['bias'].mean()
df_gr

type
blackboard     -2.410000
code           -2.886275
presentation   -1.477586
Name: bias, dtype: float64

In [13]:

bias_min = df_gr.min()

bias_min


-2.8862745098039215

In [15]:
# проверка того, что задание решено верно
assert bias_min  + 1.47 < 1e-2


__в)__ Посмотрим как оцениватель Филиппа ведёт себя в динамике. Посчитайте среднее смещение для каждой недели. Как оно ведёт себя в динамике? Правда ли, что оно постепенно сходится к нулю? Чем поведение смещения в динамике отличается от асимптотической несмещённости?

In [20]:
df_w = df.groupby(['week'])['bias'].mean()
df_w

week
1    -1.950000
2    -1.558333
3    -1.727273
4    -1.300000
5    -3.150000
6    -3.328571
7    -3.000000
8    -1.000000
9    -2.200000
10   -2.272727
11   -2.200000
Name: bias, dtype: float64

__г)__ Несмещённость - это свойство при фиксированном $n$. Состоятельность - свойство при $n \to \infty$. Представим себе, что Филипп переосмыслил свою стратегию оценки времени, которое он тратит на запись видео. Теперь он хочет оценивать время на запись следующего видео как среднее, посчитанное по всем видео, записанным ранее.

_Например:_ Филипп записал три видео:

- презентация 4 минуты
- презентация 1 минута
- доска 4 минуты

Тогда он оценить веремя на следующее видео как $\frac{4 + 1 + 4}{3} = 3$. Создайте в табличке колонку `new_estimate` с такми оценками. Время на первое видео он оценивает как $6$ минут. Посмотрите как себя ведёт такая оценка в динамике. Как думаете, к чему она сходится по вероятности? Какой физический смысл стоит за такой оценкой?

In [19]:
df

,real_week,week,type,estimate_time,real_time,bias
0,6,1,blackboard,6,11.0,-5.0
1,6,1,blackboard,10,7.0,3.0
2,6,1,blackboard,10,11.5,-1.5
3,6,1,blackboard,5,6.0,-1.0
4,6,1,presentation,5,3.0,2.0
...,...,...,...,...,...,...
124,4,11,code,10,13.0,-3.0
125,4,11,code,10,12.0,-2.0
126,4,11,code,10,14.0,-4.0
127,4,11,code,15,15.0,0.0


In [53]:
def estimate_next(row):
    # print(row.name, df.loc[:row.name, 'real_time'].mean())
    prev_mean = df.loc[:row.name, 'real_time'].mean()
    return prev_mean

df['new_estimate'] = df.apply(estimate_next, axis=1)
df.loc[0, 'new_estimate'] = 6 # для первой записи устанавливаем значение 6

df

,real_week,week,type,estimate_time,real_time,bias,new_estimate,bias_new
0,6,1,blackboard,6,11.0,-5.0,6.000000,-5.000000
1,6,1,blackboard,10,7.0,3.0,9.000000,1.000000
2,6,1,blackboard,10,11.5,-1.5,9.833333,-2.833333
3,6,1,blackboard,5,6.0,-1.0,8.875000,1.750000
4,6,1,presentation,5,3.0,2.0,7.700000,4.200000
...,...,...,...,...,...,...,...,...
124,4,11,code,10,13.0,-3.0,10.468800,-4.720000
125,4,11,code,10,12.0,-2.0,10.480952,-3.706349
126,4,11,code,10,14.0,-4.0,10.508661,-5.692913
127,4,11,code,15,15.0,0.0,10.543750,-6.640625


In [51]:
# df['new_estimate'] = df['estimate_time']
# df['new_estimate'] = df.new_estimate.expanding().mean()
# df

,real_week,week,type,estimate_time,real_time,bias,new_estimate,bias_new
0,6,1,blackboard,6,11.0,-5.0,6.000000,-5.000000
1,6,1,blackboard,10,7.0,3.0,8.000000,9.000000
2,6,1,blackboard,10,11.5,-1.5,8.666667,9.833333
3,6,1,blackboard,5,6.0,-1.0,7.750000,8.875000
4,6,1,presentation,5,3.0,2.0,7.200000,7.700000
...,...,...,...,...,...,...,...,...
124,4,11,code,10,13.0,-3.0,8.280000,10.468800
125,4,11,code,10,12.0,-2.0,8.293651,10.480952
126,4,11,code,10,14.0,-4.0,8.307087,10.508661
127,4,11,code,15,15.0,0.0,8.359375,10.543750


__д)__ Посчитайте для каждого видео смещение. По аналогии с пунктом __а)__ найдите среднее смещение. Запишите его в переменную `bias_mean_new`. Стало ли оно меньше благодаря новому оценивателю? Почему?

In [54]:
df['bias_new'] = df['new_estimate'] - df['real_time']

bias_mean_new = df['bias_new'].mean()
bias_mean_new

-0.6129047330340913

In [55]:
# проверка того, что задание решено верно
assert np.abs(bias_mean_new  + 0.609) < 1e-2


__е)__ Как Филипп мог бы использовать полученную в этом задании информацию, __чтобы стать лучше__? Сформулируйте для Филиппа несколько дельных советов.